In [21]:
import pandas as pd
import requests
import numpy as np

In [73]:
def parcours_profondeur(graphe: dict, debut: int) -> list:
    
    cycle = False
    
    taille = len(graphe.keys())
    
    etat = [None for _ in range(taille)]
    pere = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    
    nb_vu = 1
    
    pile = [debut]
    etat[debut - 1] = 0
    
    while nb_vu <= taille:
        if etat[nb_vu - 1] is not None:
            nb_vu += 1
        elif pile == []:
            pile.append(nb_vu)
        while pile != []:
            suivant = pile.pop()
            if etat[suivant - 1] is None:
                etat[suivant - 1] = 0
                pere[suivant - 1] = dernier_pred[suivant - 1]
                for successeur in graphe[suivant]:
                    pile.append(successeur)
                    if (etat[successeur - 1] is not None) and (successeur != dernier_pred[suivant - 1]):
                        cycle = True
                    dernier_pred[successeur - 1] = suivant
    return pere

def excentricite_sommet(graphe: dict, debut: int) -> list:
    
    taille = len(graphe.keys())
    
    etat = [None for _ in range(taille)]
    hauteur = [0 for _ in range(taille)]
    
    file = [debut]
    etat[debut - 1] = 0
    
    nb_vu = 1
    
    while nb_vu <= taille:
        if etat[nb_vu - 1] is not None:
            nb_vu += 1
        elif file == []:
            file.append(nb_vu)
            etat[nb_vu - 1] = 0
        while file != []:
            suivant = file[0]
            file = file[1:] if len(file) > 1 else []
            for successeur in graphe[suivant]:
                if etat[successeur - 1] is None:
                    file.append(successeur)
                    etat[successeur - 1] = 0
                    hauteur[successeur - 1] = hauteur[suivant - 1] + 1
                    
    return max(hauteur)

"""Articulation => Médiateurs, Centres => ?, Degre, Diamètre, Graphe complémentaire et stable => Protéine localisée !=
Traité les composantes connexes séparemment, Isthme => ?"""

In [2]:
proteome = pd.read_csv("Saccharomyces cerevisiae/transcriptome.tsv", sep="\t")
proteome = proteome[proteome["Gene Type"] == "protein-coding"]
proteome

,Accession,Begin,End,Chromosome,Orientation,Name,Symbol,Gene ID,Gene Type,Transcripts accession,Protein accession,Protein length,Locus tag
0,NC_001133.9,1807,2169,I,minus,seripauperin PAU8,PAU8,851229,protein-coding,NM_001180043.1,NP_009332.1,120.0,YAL068C
1,NC_001133.9,2480,2707,I,plus,uncharacterized protein,NaN,1466426,protein-coding,NM_001184582.1,NP_878038.1,75.0,YAL067W-A
2,NC_001133.9,7235,9016,I,minus,putative permease SEO1,SEO1,851230,protein-coding,NM_001178208.1,NP_009333.1,593.0,YAL067C
3,NC_001133.9,11565,11951,I,minus,uncharacterized protein,NaN,851232,protein-coding,NM_001179897.1,NP_009335.1,128.0,YAL065C
4,NC_001133.9,12046,12426,I,plus,uncharacterized protein,NaN,851233,protein-coding,NM_001180042.1,NP_009336.1,126.0,YAL064W-B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,NC_001224.1,48901,50097,MT,plus,mitochondrial 37S ribosomal protein VAR1,VAR1,854586,protein-coding,NaN,NP_009320.1,398.0,Q0140
6445,NC_001224.1,61022,61729,MT,plus,intron-encoded endonuclease I-SceI,SCEI,854590,protein-coding,NaN,NP_009324.1,235.0,Q0160
6462,NC_001224.1,73758,74513,MT,plus,cytochrome c oxidase subunit 2,COX2,854622,protein-coding,NaN,NP_009326.1,251.0,Q0250
6463,NC_001224.1,74495,75984,MT,plus,maturase-like protein,NaN,854623,protein-coding,NaN,NP_009327.1,472.0,Q0255


In [3]:
print(proteome["Locus tag"].isna().sum())

0


In [20]:
url = lambda tag, k: f"http://webservice.thebiogrid.org/interactions?geneList={tag}&format=json&taxId=559292&accesskey={k}"
cle = "47c1bde1e3d0203e63756e71e9ca4f81"
reponse = requests.get(url("Q0275", cle)).json()
print(reponse["2513187"]["SYSTEMATIC_NAME_A"], len(reponse.keys()))

# vérifier avec biogrid en passant l'id 854627

YNR020C 78


In [37]:
# +40 min de lancement faites attention
with open("Saccharomyces cerevisiae/interactome.txt", "w") as filout, open("Saccharomyces cerevisiae/non_reference.txt", "w") as fil:
    texte = "PID(1);PID(2)\n"
    textref = ""
    for pid in proteome["Locus tag"]:
        reponse = requests.get(url(pid, cle)).json()
        try:
            ss_texte = f"{pid};"
            ss_texte += f"{pid};".join(list(map(lambda x: x + "\n", [reponse[interaction]["SYSTEMATIC_NAME_A"] if reponse[interaction]["SYSTEMATIC_NAME_A"] != pid else reponse[interaction]["SYSTEMATIC_NAME_B"] for interaction in reponse.keys()]))
    )
            texte += ss_texte
        except:
            textref += f"{pid}\n"
    filout.write(texte)
    fil.write(textref)

In [38]:
#18h10 -> 19h18

In [32]:
indices = {}
noms = {}
G = {}

for i, pid in enumerate(proteome["Locus tag"]):
    G[i+1] = set()
    indices[pid] = i+1
    noms[i+1] = pid

with open("Saccharomyces cerevisiae/interactome.txt", "r") as fil:
    fil.readline()
    for ligne in fil:
        v1, v2 = ligne[:-1].split(";")
        if (v1 in indices) and (v2 in indices):
            G[indices[v1]].add(indices[v2])
            G[indices[v2]].add(indices[v1])
         
        
nb_noeuds = len(G.keys())
nb_aretes = sum(list(map(len, list(G.values()))))

In [33]:
print(nb_noeuds, nb_aretes)

6014 1216256
